In [120]:
%matplotlib inline
import matplotlib.pyplot as plt
import os
import pandas as pd
import nltk
import spacy
import glob
import numpy as np
import csv
import re
import pickle
import itertools

from sklearn import preprocessing
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer, TfidfTransformer
from sklearn.cluster import KMeans
from sklearn.linear_model import LinearRegression
from sklearn.cluster import DBSCAN
from sklearn.multiclass import OneVsRestClassifier
from sklearn.svm import LinearSVC
from sklearn.decomposition import SparsePCA
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

from sklearn.model_selection import train_test_split
from sklearn import metrics
from nltk.stem.snowball import SnowballStemmer

In [ ]:
import glob
import csv
import re

with open("Schadenshergang.csv", "a") as csvfile:
    writer = csv.writer(csvfile, delimiter=";")
    writer.writerow(["Schadenshergang"])
    for f in glob.iglob("../Inputs/Schadenmeldungen/*.txt"):
        with open(f) as file:
            text = file.read()
            m = re.search("(?<=Schadenshergang:).*", text)
            writer.writerow([m.group(0)])

In [117]:
data = pd.read_csv("Schadenshergang.csv", delimiter=";", header=0)
print(data.shape)
print(data["Schadenshergang"].iloc[0])
data.head()

(121, 1)
 Der Knetkacken ist mir beim Abtrocknen runter auf die Glaskeramikplatte gefallen.


,Schadenshergang
0,Der Knetkacken ist mir beim Abtrocknen runter...
1,Habe am 31. Juli auf der Terrasse Dachisolati...
2,Die Fäkalienpumpe im UG stellt nicht mehr ab ...
3,Beim Unwetter wurde Lamellenstore beschädigt
4,"unbekannt, wir vermuten als Ursache einer der..."


In [ ]:
def lowercase(text):
    return [w.lower() for w in text]

In [ ]:
def remove_stopwords(text):
    stopwords = nltk.corpus.stopwords.words('german')
    wordlist_stripped = [w for w in text if w.lower() not in stopwords]
    return wordlist_stripped

In [ ]:
def remove_punctuation(wordlist):
    punctuation = ['.', ',', ';', ':', '(', ')', '[', ']', '{', '}', '\"', '\'','\'\'', '\`', '\`\`', '==', '===', '====', '-']
    wordlist_stripped = [w for w in wordlist if w not in punctuation]
    wordlist_stripped = [w for w in wordlist if len(w) > 2]
    return wordlist_stripped

In [ ]:
def get_word_list_from_schadenmeldung(text):
    print(text)
    wordlist = nltk.word_tokenize(text)
    wordlist = lowercase(wordlist)
    wordlist = remove_stopwords(wordlist)
    wordlist = remove_punctuation(wordlist)
    return wordlist

In [141]:
nlp = spacy.load("de")
words = []
for k, v in data.iterrows():
    doc = nlp(v["Schadenshergang"])
    for t in doc:
        # print(t.orth_ + " " + t.pos_ + " " + str(t.head.is_stop))
        if t.pos_ == "NOUN" and t.orth_ not in words:
            words.append(t.orth_)
            
            
pd.DataFrame(words).head()
print(words)
                
with open("Corpus/gvb_corpus.txt", "a") as textFile:
    for w in words:
        textFile.writelines(w + "\n")

['Knetkacken', 'Abtrocknen', 'Glaskeramikplatte', 'Habe', 'Juli', 'Terrasse', 'Dachisolationsfetzen', 'Eingang', 'Marder', 'Fäkalienpumpe', 'Toilette', 'Studio', 'Wasserlache', 'Gerät', 'Fa.', 'AG', 'Sanitär', 'Sanitätshop.de', 'Unwetter', 'Lamellenstore', 'Ursache', 'Winterstürme', 'Datum', 'Schätzung', 'Fassade', 'Liegenschaft', 'Graffiti', 'Feuerwerkskörper', 'Sonnenstore', 'Sturm', 'Ziegel', 'Gewitter', 'Ortladen', 'Aushärtungsphase', 'Futtertischbeschichtung', 'Wassereinbruch', 'Wassereintritt', 'Waschküche', 'Schimmel', 'Keller', 'Brand', 'Umbau', 'Sanierung', 'Saal', 'Pflastermulde', 'Holz', 'Feuer', 'Sturmwind', 'Welleternit-Dach', 'Hagelereignis', 'Blumentrog', 'Parkplatz', 'Dienstag', 'Sintflutartigen', 'Regenfällen', 'Teil', 'Kellers', 'Dachdecker', 'Situation', 'Dachkanals', 'Rohrreinigungsfirma', 'Kanal', 'Dachwassers', 'Firma', 'Flückiger', 'Nachmittag', 'Wunsch', 'Ablauf', 'Leitung', 'Tagen', 'Kamera', 'Schäden', 'Kamera-Aufzeichnung', '31.07.2018', 'Termin', '31.07.18',

In [ ]:
gvb_korpus = nltk.corpus.PlaintextCorpusReader("Corpus/", ".*")
words = nltk.FreqDist(gvb_korpus.words())
count = sum(words.values())
vocab = len(words)

print(count)
print(vocab)

gvb_word_list = [w for w in gvb_korpus.words()]
matches = []
for w in gvb_word_list:
    if w in data["Schadenshergang"].iloc[1] and w not in matches:
        matches.append(w)
        
for m in matches:
    print(m)

In [143]:
with open("Schadenshergang_categorize.csv", "a") as csvfile:
    writer = csv.writer(csvfile, delimiter=";")
    writer.writerow(["Schadenshergang", "Objekte", "Schadensursache"])
    for f in glob.iglob("../Inputs/Schadenmeldungen/*.txt"):
        with open(f) as file:
            text = file.read()
            m_hergang = re.search("(?<=Schadenshergang:).*", text)
            m_object_geraete = re.search("(?<=Beschädigte Geräte:).*", text)
            m_object_umgebung = re.search("(?<=Beschädigte Umgebung:).*", text)
            m_object_gebaeude = re.search("(?<=Beschädigte Gebäudeteile:).*", text)
            m_ursache = re.search("(?<=Schadensursache:).*", text)
            writer.writerow([m_hergang.group(0).lstrip(), m_object_geraete.group(0).lstrip() + " " + m_object_umgebung.group(0).lstrip() + " " + m_object_gebaeude.group(0).lstrip(), m_ursache.group(0).lstrip()])

In [144]:
csv_cat = pd.read_csv("Schadenshergang_categorize.csv", delimiter=";", header=0)
csv_cat = csv_cat.dropna()
X = csv_cat.drop(columns=["Schadensursache"])
y = csv_cat["Schadensursache"]

In [161]:
stemmer = SnowballStemmer("german")
stopwords = nltk.corpus.stopwords.words('german')
def preprocess_text(text):
    tokens = [word for sent in nltk.sent_tokenize(text) for word in nltk.word_tokenize(sent) if word not in stopwords]
    filtered_tokens = []
    for token in tokens:
        if re.search('[a-zA-Z]', token):
            filtered_tokens.append(token)
    stems = [stemmer.stem(t) for t in filtered_tokens]
    return stems

In [162]:
tfidf_vectorizer = TfidfVectorizer(tokenizer=preprocess_text)
tfidf_matrix = tfidf_vectorizer.fit_transform(csv_cat["Schadenshergang"] + " " + csv_cat["Objekte"])
print(tfidf_matrix.shape)
print(tfidf_vectorizer.get_feature_names())

(121, 824)
['-etwa', '0015up', '1x', '2342797-a', '2342797-b', '2x', '601fu', 'ab', 'abdeck', 'abfalleim', 'abfliess', 'abgefloss', 'abgemacht', 'abgepumpt', 'ablauf', 'abtransporti', 'abtrockn', 'abwass', 'abwasserrohr', 'abwasserschacht', 'abwes', 'ag', 'alarmiert', 'alt', 'altbau', 'ameisenbefall', 'anbau', 'andauert', 'anfang', 'anfrag', 'angebracht', 'angehangt', 'angeschlag', 'anhang', 'anschluss', 'art', 'au', 'aufgebot', 'aufgebroch', 'aufgedruckt', 'aufgefordert', 'aufgespitzt', 'aufgestaut', 'aufgetret', 'aufgrund', 'aufmerksam', 'aufsaug', 'auftrag', 'aufzunehm', 'august', 'august-', 'ausgeriss', 'ausgeschnitt', 'ausgestieg', 'ausgetret', 'ausgewechselt', 'aushartungsphas', 'auslauf', 'auspumpen/trockn', 'auss', 'aussenschal', 'aussentrepp', 'aussergewohn', 'autobahnuberfuhr', 'automat', 'bach', 'backofen/', 'bad', 'badewann', 'badezimm', 'badzimm', 'balkon', 'balkonbod', 'ball', 'bauernhaus', 'bedach', 'bedeckt', 'bedekt', 'beer', 'begleitet', 'beim', 'bekannt', 'belag', 'b

In [167]:
def cluster_metric(data, true_labels, cluster_labels):
    print()
    print("Cluster Metric:")
    print("Homogeneity: ", metrics.homogeneity_score(true_labels, cluster_labels))
    print("Completeness: ", metrics.completeness_score(true_labels, cluster_labels))
    print("V-Measure: ", metrics.v_measure_score(true_labels, cluster_labels))
    print("Silhouette Score: ", metrics.silhouette_score(data, cluster_labels, metric='euclidean'))
    print()

In [ ]:
def visualize_k_means(data, k_means):
    plt.figure(figsize=(8,6), dpi=120)
    k = len(k_means.cluster_centers_)
    for i in range(k):
        colors = ['r', 'g', 'b', 'y', 'c', 'm']
        points_in_cluster = np.array([data[j] for j in range(len(data)) if k_means.labels_[j] == i])
        plt.scatter(points_in_cluster[:, 0], points_in_cluster[:, 1], \
                    marker='.', color=colors[i % len(colors)])
    plt.scatter(k_means.cluster_centers_[:,0], k_means.cluster_centers_[:, 1], marker='*', color='k')
    plt.title('k-Means Clustering with k={} clusters'.format(k))
    plt.show()

In [77]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [168]:
k = 25
k_means = KMeans(n_clusters=k, max_iter=100, n_init=10, init="k-means++").fit(tfidf_matrix)
clusters = k_means.labels_.tolist()

print("Top terms per cluster:")
print()
terms = tfidf_vectorizer.get_feature_names()
order_centroids = k_means.cluster_centers_.argsort()[:, ::-1]
for i in range(k):
    print("Cluster {}".format(i))
    for ind in order_centroids[i, :10]:
        print(terms[ind])
    print()
    
cluster_metric(tfidf_matrix, y, k_means.labels_)

Top terms per cluster:

Cluster 0
kell
kellerraum
ruckstau
wassereintritt
stark
wass
tiefkuhlschrank
unwett
entdeckt
dachkanel

Cluster 1
miet
verglas
herr
verschraub
nr
gerat
tel
brian
gemeldet
beim

Cluster 2
fassad
versprayt
spayerei
sprayerei
spreyerei
isolation
wurd
graffiti
liegenschaft
fassadenteil

Cluster 3
wc
mau
farbe-verputz
kamin
druckt
rohrbruch
abwass
dass
wass
ergeb

Cluster 4
lamellenstor
bodenplatt
balkon
beschadigt
wohnung
wurd
grill
recht
dg
beim

Cluster 5
sonnenstor
knickarm
windbo
beschadigt
gebroch
gross
wespenn
entdecket
glasdach
sonnenstorenstoff

Cluster 6
deck
wand
undicht
kuch
bodenablauf
einstellhall
wass
erst
fussleist
bod

Cluster 7
nein
dachziegel
dach
glasscheib
hagelschad
wintergart
famili
beer
beschadigt
gelost

Cluster 8
haustur
wassereinbruch
hornissenn
wasserunterbruch
glasbruch
insekt
belast
glasscheib
rasenflach
versprayt

Cluster 9
glaskeramikplatt
gefall
waschtisch
beim
knetkack
abtrockn
runt
keram
handewasch
minut

Cluster 10
stor
wespenn
dre

In [169]:
pca = SparsePCA(n_components=2)
tfidf_matrix_pca = pca.fit_transform(tfidf_matrix.toarray())
print(tfidf_matrix_pca.shape)

k = 25
k_means = KMeans(n_clusters=k, max_iter=100, n_init=10, init="k-means++").fit(tfidf_matrix_pca)
cluster_metric(tfidf_matrix_pca, y, k_means.labels_)

(121, 2)

Cluster Metric:
Homogeneity:  0.15723956413832907
Completeness:  0.4250479482253596
V-Measure:  0.22955791665716782
Silhouette Score:  0.8512396694214877



In [145]:
csv_cat = pd.read_csv("Schadenshergang_categorize.csv", delimiter=";", header=0)
X = csv_cat.drop(columns=["Schadensursache"])
y = csv_cat["Schadensursache"]

test = []
for k, v in X.iterrows():
    test.append(v["Schadenshergang"] + " " + v["Objekte"])

X_train, X_test, y_train, y_test = train_test_split(test, y, test_size=0.3, random_state=0)

In [146]:
model = Pipeline([('vectorizer', CountVectorizer(ngram_range=(1,2))),
 ("tfidf", TfidfTransformer(use_idf=True)),
 ("clf", OneVsRestClassifier(LinearRegression()))])

In [147]:
model.fit(X_train, y_train)
pred = model.predict(X_test)
confusion_matrix(pred, y_test)
print("Test Score: {:.3%}\n".format(model.score(X_test, y_test)))
print(classification_report(y_test, pred))

Test Score: 45.946%

                                       precision    recall  f1-score   support

                          Blitzschlag       1.00      0.50      0.67         2
                                Feuer       0.00      0.00      0.00         2
              Gebäudehaftpflichtsfall       0.00      0.00      0.00         1
                       Gebäudetechnik       0.00      0.00      0.00         3
                            Glasbruch       0.00      0.00      0.00         0
                    Grund-/Hangwasser       0.00      0.00      0.00         1
                                Hagel       0.75      0.75      0.75         4
                           Hochwasser       0.00      0.00      0.00         1
                        Leitungsbruch       0.33      0.33      0.33         3
 Marder-, Nager- oder Insektenschäden       0.50      0.50      0.50         4
         Regen-/Schnee-/Schmelzwasser       0.00      0.00      0.00         1
                            St

/Users/oliver.hofmann/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)
/Users/oliver.hofmann/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1137: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


In [148]:
print("Export the model to model.pkl")
f = open('model.pkl', 'wb')
pickle.dump(model, f)
f.close()

Export the model to model.pkl


In [149]:
print('Import the model from model.pkl')
f2 = open('model.pkl', 'rb')
pickel_model = pickle.load(f2)

Import the model from model.pkl


In [150]:
pickel_model.predict(["Als der Orkan Lothar, welcher über der Biskaya enstanden ist und mit einer spitzen Geschwindigkeit von 272km/h über die Schweiz fegte, fegte er auch über mein zu Hause! Dabei entwurzelte der Sturm Hecken aus dem Garten, herumfliegende Äste haben mehrere Fenster eingeschlagen, sinnflutartige Regenfälle haben meinen Keller gefüllt und Wasserschaden verursacht und eine kräftigen Windböen haben die Ziegel vom Dach gerissen."])[0]

'Sturmwind'